## Third Movement: 

In [177]:
# Import required libraries
import networkx as nx
import numpy as np
from scipy.special import factorial
from datetime import datetime
import logging
from scipy.stats import geom
import sys
import os

In [178]:
import os
experiment_log_file = 'experiment_3.log'
log_path = os.path.join(os.getcwd(), experiment_log_file)

if os.path.exists(log_path):
    os.remove(log_path)

logger = logging.getLogger()
handler = logging.FileHandler(log_path, mode='w')
handler.setFormatter(logging.Formatter('%(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info(f'[1] {experiment_log_file}')
logger.info(f'[1] "{datetime.now().strftime("%a %b %d %H:%M:%S %Y")}"')

In [179]:
%run attack_graph_MIR100.ipynb

### Use Third Distribution

In [180]:
attack_rate_list = [0]   
defense_rate_list = [0]  

In [181]:
def random_steps(route, attack_rate=None, defense_rate=None, graph=None):
    #print("\n=== Debug: random_steps function ===")
    #print("Input route:", route)
    
    hardness = []
    #print("\nCollecting edge probabilities:")
    
    for i in range(len(route) - 1):
        u, v = route[i], route[i+1]
        edges_dict = graph[u][v]  
        #print(f"\nEdge {u} -> {v}:")
        #print("Raw edge data:", edges_dict)

        if isinstance(edges_dict, float):
            min_weight = edges_dict
            #print(f"Direct float weight: {min_weight}")
        elif hasattr(edges_dict, 'items'):
            weights = []
            #print("Multiple edges found:")
            for edge_key, attr_dict in edges_dict.items():
                if isinstance(attr_dict, dict) and 'weight' in attr_dict:
                    weight = attr_dict['weight']
                    #print(f"  Edge {edge_key}: weight = {weight}")
                    weights.append(weight)
                elif isinstance(attr_dict, float):
                    #print(f"  Edge {edge_key}: direct weight = {attr_dict}")
                    weights.append(attr_dict)

            if weights:
                min_weight = min(weights) # Take minimum weight for parallel edges
                max_weight = max(weights) # Take maximum weight for parallel edges
                #print(f"Selected minimum weight: {min_weight}")
                #print(f"Selected maximum weight: {max_weight}")
            else:
                min_weight = float(edges_dict.get('weight', 0.0))
                max_weight = float(edges_dict.get('weight', 0.0))
                #print(f"Single edge weight: {min_weight}")

            # SWITCH BETWEEN THESE TWO APPROACHES:
            # Approach 1: Using minimum weight (original)
            # prob = np.exp(-min_weight)
            # Approach 2: Using maximum weight (new)
            prob = np.exp(-max_weight)

            #print(f"Converted to probability: {prob}")
            hardness.append(prob)

    #print("\nFinal hardness array:", hardness)
    
    hardness = np.array(hardness)
    hardness = np.nan_to_num(hardness, nan=1.0)
    #print("Hardness after nan handling:", hardness)

    cumulative_probs = np.concatenate(([1.0], np.cumprod(hardness)))
    #print("Cumulative probabilities:", cumulative_probs)
    
    stop_probs = 1 - hardness
    stop_probs = np.concatenate((stop_probs, [1.0]))
    #print("Stop probabilities:", stop_probs)

    pdf = cumulative_probs * stop_probs
    #print("PDF before normalization:", pdf)
    
    pdf_sum = pdf.sum()
    if pdf_sum < 1e-15:
        pdf[:] = 1e-7
        pdf_sum = pdf.sum()

    final_pdf = pdf / pdf_sum
    #print("Final normalized PDF:", final_pdf)
    #print("=== End random_steps debug ===\n")
    
    return final_pdf

In [182]:
# %run ctr-core_simple.ipynb
%run ctr-core_tests.ipynb
main()


Before merging targets:
Nodes: [1, 5, 15, 12, 11, 13, 3, 6, 8, 4, 7, 2, 9, 10, 14, 16, 0]
Edges with weights:
1 -> 5 : 2.1958405355640576
5 -> 15 : 2.1958405355640576
15 -> 12 : 0.7489220813074156
15 -> 13 : 0.7489220813074156
15 -> 16 : 0.7489220813074156
11 -> 13 : 1.064439873679208
11 -> 14 : 0.7489220813074156
11 -> 16 : 0.0
3 -> 6 : 1.064439873679208
3 -> 8 : 0.7489220813074156
6 -> 8 : 0.0
8 -> 10 : 0.0
8 -> 14 : 0.0
8 -> 16 : 0.7489220813074156
4 -> 7 : 0.7489220813074156
7 -> 10 : 0.7489220813074156
7 -> 14 : 0.7489220813074156
7 -> 16 : 0.7489220813074156
2 -> 9 : 0.7489220813074156
2 -> 10 : 1.064439873679208
2 -> 11 : 0.7489220813074156
2 -> 16 : 1.064439873679208
9 -> 14 : 1.064439873679208
10 -> 15 : 0.0
0 -> 1 : 1
0 -> 3 : 1
0 -> 4 : 1
0 -> 2 : 1

After merging targets:
Nodes: [1, 5, 15, 11, 3, 6, 8, 4, 7, 2, 9, 10, 0, 'c(12,13,14,16)']
Edges with weights:
1 -> 5 (key=0) : 2.1958405355640576
5 -> 15 (key=0) : 2.1958405355640576
15 -> c(12,13,14,16) (key=0) : 0.7489220813

In [183]:
with open(experiment_log_file, 'r') as f:
    print(f.read())

[1] experiment_3.log
[1] "Thu Jan 02 23:08:53 2025"

++++++++++++++++++++++++++++++++

The virtual target nodeID is c(12,13,14,16)

attack rate =  0 , defense rate =  0 

	equilibrium for multiobjective security game (MOSG)

optimal defense strategy:
         prob.
10 0.000000e+00
11 0.000000e+00
15 4.799774e-01
5 0.000000e+00
6 0.000000e+00
7 9.098599e-02
8 4.290366e-01
9 0.000000e+00

worst case attack strategies per goal:
          1
1 0.0000000
2 0.3278186
3 0.0000000
4 0.0000000
5 0.0000000
6 0.3871037
7 0.2850777
8 0.0000000
9 0.0000000
10 0.0000000
11 0.0000000
[1] 0.053

Defender can keep attacker success below: 0.053
Attacker can guarantee success probability of: 0.053



In [184]:
# def random_steps(route, attack_rate=None, defense_rate=None, graph=None):
#     """
#     Calculate probability distribution based on edge difficulties.
    
#     Args:
#         route: List of nodes representing an attack path
#         attack_rate: Not used in this implementation but kept for API consistency
#         defense_rate: Not used in this implementation but kept for API consistency
#         graph: NetworkX graph containing edge probabilities
        
#     Returns:
#         numpy array: Probability distribution for number of steps attacker can take
#     """
#     # Get edge probabilities between consecutive nodes in route
#     hardness = []
#     for i in range(len(route)-1):
#         # Get edge data between these nodes
#         edges_data = graph.get_edge_data(route[i], route[i+1])
        
#         # Handle both MultiDiGraph and DiGraph edge data structures
#         if isinstance(edges_data, dict):
#             if 'edge_probabilities' in edges_data:
#                 # DiGraph case - direct dictionary
#                 prob = float(edges_data.get('edge_probabilities', 1.0))
#                 hardness.append(prob)
#             else:
#                 # MultiDiGraph case - dictionary of edge dictionaries
#                 probs = []
#                 for key in edges_data:
#                     if isinstance(edges_data[key], dict):
#                         prob = edges_data[key].get('edge_probabilities', 1.0)
#                         probs.append(float(prob))
#                     else:
#                         probs.append(float(edges_data.get('edge_probabilities', 1.0)))
#                 hardness.append(max(probs) if probs else 1.0)
#         else:
#             # No edge data found
#             hardness.append(1.0)
    
#     # Convert to numpy array and handle missing values
#     hardness = np.array(hardness)
#     hardness[np.isnan(hardness)] = 1.0  # Missing probabilities treated as certainty
    
#     # Calculate PDF according to the paper's formula
#     # P(n steps) = (1-p_{n+1}) * product(p_1 to p_n)
#     # where p_i is probability of success at step i
    
#     # Calculate cumulative products
#     cumprod = np.concatenate(([1.0], np.cumprod(hardness)))
    
#     # Calculate (1-p) terms
#     one_minus_h = np.concatenate((1 - hardness, [1.0]))
    
#     # Final PDF calculation
#     pdf = one_minus_h * cumprod
    
#     # Normalize to ensure it sums to 1
#     pdf = pdf / np.sum(pdf)
    
#     return pdf